In [12]:
import pandas as pd

df_incident = pd.read_excel("CHDS_raw_Data.xlsx", sheet_name="INCIDENT")
df_victim = pd.read_excel("CHDS_raw_Data.xlsx", sheet_name="VICTIM")
df_victim.rename(columns={"incidentid": "Incident_ID"}, inplace=True)

# merge the two dataframes based on Incident_ID
merged_df = pd.merge(df_incident, df_victim, on="Incident_ID")

# filter the merged dataframe to only include Fatal injuries
fatal_df = merged_df[merged_df["injury"] == "Fatal"]

# group the filtered dataframe by Incident_ID and count the number of rows
fatal_count_df = fatal_df.groupby("Incident_ID").size().reset_index(name="Number Killed")

# merge the fatal count dataframe with the original dataframe
result_df = pd.merge(df_incident, fatal_count_df, on="Incident_ID")

# print the result dataframe
df = result_df[["Incident_ID", "Date", "School", "Number Killed", "City", "State"]]


In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# create a new dataframe that counts the number of deaths per incident
# deaths_df = df.groupby(["School"]).agg({"Number Killed": "sum"}).reset_index()

df["Year"] = pd.to_datetime(df["Date"]).dt.year
df = df.sort_values("Number Killed", ascending=True)
max_killed =  df["Number Killed"].max()
#df["Group"] = df.groupby("Year")["Number Killed"].apply(lambda x: ["Highest" if i == x.max() else "Other" for i in x])
for year in df["Year"].unique():
    year_mask = df["Year"] == year
    max_killed = df[year_mask]["Number Killed"].max()
    df.loc[year_mask, "Group"] = ["Highest" if x == max_killed else "Other" for x in df[year_mask]["Number Killed"]]
    if df.loc[year_mask, "Group"].nunique() == 1:
        df.loc[year_mask, "Group"] = "Other"

df["marker_color"] = ["red" if g == "Highest" else "lightpink" for g in df["Group"]]
#colors = ["lightgrey"] * (len(df) - 1) + ["red"]

# create the stacked bar chart with hover tooltips
fig = go.Figure(go.Bar(x=df["Year"], y=df["Number Killed"], customdata=df[['School', 'City', 'State']].values.tolist(),
             hovertemplate="<b>School: %{customdata[0]}</b><br><b>Deaths: %{y}</b><br><b>Location: %{customdata[1]}, %{customdata[2]}</b><br> <extra></extra>", marker_color=df["marker_color"]))
fig.update_layout(title="Fatalities in School Shootings (1970-2022)",
                  xaxis_title="Year",
                  yaxis_title="Number of Deaths",
                  hoverlabel=dict(bgcolor='lightgray', font=dict(color='Black')),
                  plot_bgcolor='white',
                  )
fig.add_annotation(x="2012", y=26, text="26 Killed at Sandy Hook <br> Elementary School", showarrow=True, yshift=50, arrowhead=1)
fig.show()
fig.write_html("incidents_binned.html")


/var/folders/9m/yhr_gxtd03q9rz3gdlc3ctqw0000gn/T/ipykernel_29361/4074538039.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

